In [47]:
import requests
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from CertificationAuthority import CertificationAuthority
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
import util

# get all ca cert and public key

In [49]:
ca_ports={}
ca_ports['ca2_1']=8004
ca_ports['ca1_1']=8003
ca_ports['ca2']=8002
ca_ports['ca1']=8001
ca_ports['ca_root']=8000
ca_datas={} # store public key and cert
def get_all_ca_pub_keys():
    for name,port in ca_ports.items():
        response= requests.get(f"http://127.0.0.1:{port}/get_CA_public_key")
        pub_key=serialization.load_pem_public_key(response.content)
        ca_datas[name]={}
        ca_datas[name]["public_key"]=pub_key
        response= requests.get(f"http://127.0.0.1:{port}/get_CA_cert")
        cert=x509.load_pem_x509_certificate(response.content)
        ca_datas[name]['cert']=cert
get_all_ca_pub_keys()
for k,v in ca_datas.items():
    print(k+": "+str(v))

    


ca2_1: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fbc8c0b5d50>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca2_1, 2.5.4.3=ca2_1, )>, ...)>}
ca1_1: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fbc8c0b55d0>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca1_1, 2.5.4.3=ca1_1, )>, ...)>}
ca2: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fbc8c0f6230>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca2, 2.5.4.3=ca2, )>, ...)>}
ca1: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fbc8c0b6b00>, 'cert': <Certificate(subject=<Name(2.5.4.6=SG, 2.5.4.8=Singapore, 2.5.4.7=Singapore, 2.5.4.10=ca1, 2.5.4.3=ca1, )>, ...)>}
ca_root: {'public_key': <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey obje

![title](https://user-images.githubusercontent.com/7097606/143174026-dfc7c921-7582-4cf2-82c8-0cfbc98232a7.png)
# root CA use self sign cert
# intermediate will get cert from 1 level above

## create certificate sign request and get sign for 
- client 1 sign by ca1_1
- client 2 sign by ca2_1

In [50]:
#ca_root use port 8000
#ca1 use port 8001
#ca2 use port 8002
#ca1_1 use port 8003
#ca1_2 use port 8004

# cleint 1 cert sign by root
# 1. generate RSA key
rsa_key=util.generate_ras_key()

# 2. create certificate sign request and sign with client private key
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"client1.com"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"mysite.com"),
        x509.DNSName(u"www.mysite.com"),
        x509.DNSName(u"subdomain.mysite.com"),
    ]),
    critical=False,
# Sign the CSR with our private key. is applicant's private key
).sign(rsa_key, hashes.SHA256())

# 3. send to certificate authority,  the data param name corresond fastapi parameter
response = requests.post(f"http://127.0.0.1:{ca_ports['ca1_1']}/issue_cert",files={'csr_file':csr.public_bytes(serialization.Encoding.PEM)})
client1_cert = x509.load_pem_x509_certificate(response.content)
print(response.content)
print("applicant: "+client1_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+client1_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client1_cert.not_valid_before)
print(client1_cert.not_valid_after)

b'-----BEGIN CERTIFICATE-----\nMIIDRjCCAi6gAwIBAgIUUCdojaLKe3CLUVQTFrNGlAQ97DYwDQYJKoZIhvcNAQEL\nBQAwVTELMAkGA1UEBhMCU0cxEjAQBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJ\nU2luZ2Fwb3JlMQ4wDAYDVQQKDAVjYTFfMTEOMAwGA1UEAwwFY2ExXzEwHhcNMjEx\nMTI3MDgwOTM0WhcNMjExMjA3MDgwOTM0WjBlMQswCQYDVQQGEwJVUzETMBEGA1UE\nCAwKQ2FsaWZvcm5pYTEWMBQGA1UEBwwNU2FuIEZyYW5jaXNjbzETMBEGA1UECgwK\nTXkgQ29tcGFueTEUMBIGA1UEAwwLY2xpZW50MS5jb20wggEiMA0GCSqGSIb3DQEB\nAQUAA4IBDwAwggEKAoIBAQDMViQz/xSrURSJK1OwgHcQXVUqXRJ6Brq7tzlWtbdY\nUtsJick0CBZgmjKMxFQsJL9Tv81erlo1Fw5UDcwgQAT5HlU/X60q6Z5hFvmzR9D1\nFJbAyiZIiSYo3JX5RvpYAzx4zUbFwOFSjEfmm9tu+LVUzgzScIOEpF2LtzVtrIum\neg6rrh7mci2AzvhZ6x50+2daAPjNlQn9DTz649s9Sq/fZzyCS8N/pghyDiXq2S6W\nzXrH2d1VraywVmpwX6xpZbDnHOH2tYIWzvkRjUjRnw1ak/4fnf9SGnPtyrqP8GUD\nkPfKFEzH3o1sTXmxBLdSEc8PXt5AOE0wuB3qMHMd5Yb7AgMBAAEwDQYJKoZIhvcN\nAQELBQADggEBAK8jAf2G1XVxwUMLyJajZxLeUhPRdndoBUv4f0o6oH1iuzRMCoad\nI0zziie5gZqDIXhPB8DXJE+md09BtHqZP4AAfliur0+j/BVVyuvkDeGlBd15vBzD\nzQNVyzADRKVqc6mloTUkvuu9QkC47v7DJ0i9N5K7Bezvx

In [51]:
# cleint 2 cert sign by ca2_1
# 1. generate RSA key
rsa_key=util.generate_ras_key()

# 2. create certificate sign request and sign with client private key
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"client2.com"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"mysite.com"),
        x509.DNSName(u"www.mysite.com"),
        x509.DNSName(u"subdomain.mysite.com"),
    ]),
    critical=False,
# Sign the CSR with our private key. is applicant's private key
).sign(rsa_key, hashes.SHA256())

# 3. send to certificate authority,  the data param name corresond fastapi parameter
## notice here port is 8001
response = requests.post(f"http://127.0.0.1:{ca_ports['ca2_1']}/issue_cert",files={'csr_file':csr.public_bytes(serialization.Encoding.PEM)})
client2_cert = x509.load_pem_x509_certificate(response.content)
print(response.content)
print("applicant: "+client2_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+client2_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client2_cert.not_valid_before)
print(client2_cert.not_valid_after)

b'-----BEGIN CERTIFICATE-----\nMIIDRjCCAi6gAwIBAgIUaM2DRQt6XuwWpyfLuyPHgW4QGI8wDQYJKoZIhvcNAQEL\nBQAwVTELMAkGA1UEBhMCU0cxEjAQBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJ\nU2luZ2Fwb3JlMQ4wDAYDVQQKDAVjYTJfMTEOMAwGA1UEAwwFY2EyXzEwHhcNMjEx\nMTI3MDgwOTQ4WhcNMjExMjA3MDgwOTQ4WjBlMQswCQYDVQQGEwJVUzETMBEGA1UE\nCAwKQ2FsaWZvcm5pYTEWMBQGA1UEBwwNU2FuIEZyYW5jaXNjbzETMBEGA1UECgwK\nTXkgQ29tcGFueTEUMBIGA1UEAwwLY2xpZW50Mi5jb20wggEiMA0GCSqGSIb3DQEB\nAQUAA4IBDwAwggEKAoIBAQDcOcFWVsOLgXsc2392qO7BEwJ+13z2/mq2FkFxq5uR\nYwEhcaq+gRlbO0e863SuwajiQGGcejm6eXwqjcGGLcpizQWsFmDHL9K0/vnH9Iu6\nADZiHjx1YCWszXnE3Z6HsVvLxEVeiA/FXw0V+jj3Z5aI95g5QWwc1JbrnQAU8F6X\nZRcqIhpOguy42rE298Knqnk8XbqE9DcC0P6LS/O26XQBucd1FIlzVidmB1Z2Ksyn\nmU3WPTjyBsnxj5v/uqSGzMflxpFPhbe1KghYXlf1emn7CybguoZ0+tG8En8OtCgm\nfrqD7wpf/93/f06ofyftBOl24/YApJ0mInuhp3KpmvlZAgMBAAEwDQYJKoZIhvcN\nAQELBQADggEBAJCGgvqLi1gQ43/0/LnDfwvq+rOGDTwtqcKvjZ/4QF92+pQrp5c5\nIwXztIGPcOD41C67tdDm0JCq9TFRHgmMQTZvrprD/9BemXE/JJGEdXnt/RhQ4bN8\ndG+OO8QBklckau5jj2wN6xhmiyBcI5pwvRGNSFQZoNZdi

In [52]:
#verify chain
def verify_chain(cur_cert):
    while True:
        #eg, ca1_1
        subjective=cur_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value
        issuer_commonName=cur_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value
        print(f"\nsubjective: {subjective} \t issuer: {issuer_commonName}")
        print(f"====check revoke status for {subjective} from {issuer_commonName}")
        #print(f"http://127.0.0.1:{ca_ports[issuer_commonName]}/revoke_cert_status")
        response = requests.get(f"http://127.0.0.1:{ca_ports[issuer_commonName]}/revoke_cert_status",files={'crt_file':cur_cert.public_bytes(serialization.Encoding.PEM)})
        print(response.content)
        print(f"=====verfy cert using {issuer_commonName} public key")
        pub_key=ca_datas[issuer_commonName]['public_key']
        status=util.verify_cert_signature(cur_cert,pub_key)
        if status:
            print("correct signature")
        else:
            print("XXX wrong signaure XXX")
        if issuer_commonName=='ca_root':
            break
        
        cur_cert=ca_datas[issuer_commonName]['cert']
verify_chain(client1_cert)


subjective: client1.com 	 issuer: ca1_1
====check revoke status for client1.com from ca1_1
b'{"msg":"Not revoke"}'
=====verfy cert using ca1_1 public key
correct signature

subjective: ca1_1 	 issuer: ca1
====check revoke status for ca1_1 from ca1
b'{"msg":"Not revoke"}'
=====verfy cert using ca1 public key
correct signature

subjective: ca1 	 issuer: ca_root
====check revoke status for ca1 from ca_root
b'{"msg":"Not revoke"}'
=====verfy cert using ca_root public key
correct signature


# Revoke client 2 from ca1_1, will fail because not issue by it

In [53]:
# revoke client2 cert from ca1_1, will fail because not issue by it
cpb=client2_cert.public_bytes(serialization.Encoding.PEM)
response = requests.post(f"http://127.0.0.1:{ca_ports['ca1_1']}/revoke_cert",files={'crt_file':cpb})
print(response.content)


b'{"msg":"The certificate was not issue by this server, the issuer is ca2_1"}'


# Revoke client 1 from ca1_1 twice, return 2 different message

In [55]:
# revoke client1 cert from ca1_1
cpb=client1_cert.public_bytes(serialization.Encoding.PEM)
print(f"http://127.0.0.1:{ca_ports['ca1_1']}/revoke_cert")
response = requests.post(f"http://127.0.0.1:{ca_ports['ca1_1']}/revoke_cert",files={'crt_file':cpb})
print(response.content)

http://127.0.0.1:8003/revoke_cert
b'{"msg":"Already revoke"}'


In [56]:
verify_chain(client1_cert)


subjective: client1.com 	 issuer: ca1_1
====check revoke status for client1.com from ca1_1
b'{"msg":"Already revoke"}'
=====verfy cert using ca1_1 public key
correct signature

subjective: ca1_1 	 issuer: ca1
====check revoke status for ca1_1 from ca1
b'{"msg":"Not revoke"}'
=====verfy cert using ca1 public key
correct signature

subjective: ca1 	 issuer: ca_root
====check revoke status for ca1 from ca_root
b'{"msg":"Not revoke"}'
=====verfy cert using ca_root public key
correct signature


# revoke ca2 cert, hierarchy

In [57]:
verify_chain(client2_cert)


subjective: client2.com 	 issuer: ca2_1
====check revoke status for client2.com from ca2_1
b'{"msg":"Not revoke"}'
=====verfy cert using ca2_1 public key
correct signature

subjective: ca2_1 	 issuer: ca2
====check revoke status for ca2_1 from ca2
b'{"msg":"Not revoke"}'
=====verfy cert using ca2 public key
correct signature

subjective: ca2 	 issuer: ca_root
====check revoke status for ca2 from ca_root
b'{"msg":"Not revoke"}'
=====verfy cert using ca_root public key
correct signature


In [58]:
# get ca cert from ca2, the original cert
response = requests.get(f"http://127.0.0.1:{ca_ports['ca2']}/get_CA_cert")
print(response.content)

b'-----BEGIN CERTIFICATE-----\nMIIDNjCCAh6gAwIBAgIUcRJxCqJG+NyVh1sBcmr+5p1D5mkwDQYJKoZIhvcNAQEL\nBQAwWTELMAkGA1UEBhMCU0cxEjAQBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJ\nU2luZ2Fwb3JlMRAwDgYDVQQKDAdjYV9yb290MRAwDgYDVQQDDAdjYV9yb290MB4X\nDTIxMTEyNzA4MDc1NloXDTIxMTIwNzA4MDc1NlowUTELMAkGA1UEBhMCU0cxEjAQ\nBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJU2luZ2Fwb3JlMQwwCgYDVQQKDANj\nYTIxDDAKBgNVBAMMA2NhMjCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEB\nAOlmKMYghsX0tcJslz9Gca6rz3c1MSZCPGHTFcrNRAGZ9O8P5cQNk8heHccIk0G0\ndpbxTR78DEA0jUwTlpWVPACAQlXcDwi/cjbpCN4jgeU8rMaGJ6hsSBK6OadWD5pV\ns7LyP/mejyHR9DzM00nHuChD1/5PbirPIJVybaxNSAt8Lp9HhP1XZ0MNuGaD9T7Q\nGuvNnFLhsTVv546C7Rp1+P4Afi4jSz2ieDVUtqxQ30w5rOGRuPmsx27D/yqd68Lt\nENFmaivByh431XDjUj4N81rhULfWTa0emYkkuUKzeyHIyZnbfzneTdtNr+QS4SZE\n/Ken/iIJ3Bd+Tp+7rGYo55cCAwEAATANBgkqhkiG9w0BAQsFAAOCAQEARumHmLmD\nnrz2fIR9H4oY9qnEuYDfhkBMKmAAk6/xe6US/e0ds0HI+obe2S+r5iGi5R6rSuQL\nk+bjI2iKrjTQtZZ0i1THLJdhKtL6SWR9GdTB+iNeZOCWhTE0BspuwBzdnSCU6AUP\n4MydR9CgZaINXYJl52KjMBZ0bBPXDx0/6+tD0XsVqq9kZ

In [59]:
# revoke ca2 cert from ca_root
response=requests.post(f"http://127.0.0.1:{ca_ports['ca2']}/revoke_ca_cert")
print(response.content)

b'{"msg":"CA cert revoked, will get new cert on next request"}'


In [60]:
# verify again, get ca cert from ca2, the cert will different, this api will always return a cert, if delete, it will create a new one
response = requests.get(f"http://127.0.0.1:{ca_ports['ca2']}/get_CA_cert")
print(response.content)

b'-----BEGIN CERTIFICATE-----\nMIIDNjCCAh6gAwIBAgIUTlciJMHymFG60jUpyZRV7rWnflwwDQYJKoZIhvcNAQEL\nBQAwWTELMAkGA1UEBhMCU0cxEjAQBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJ\nU2luZ2Fwb3JlMRAwDgYDVQQKDAdjYV9yb290MRAwDgYDVQQDDAdjYV9yb290MB4X\nDTIxMTEyNzA4MTIxNVoXDTIxMTIwNzA4MTIxNVowUTELMAkGA1UEBhMCU0cxEjAQ\nBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJU2luZ2Fwb3JlMQwwCgYDVQQKDANj\nYTIxDDAKBgNVBAMMA2NhMjCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEB\nAOlmKMYghsX0tcJslz9Gca6rz3c1MSZCPGHTFcrNRAGZ9O8P5cQNk8heHccIk0G0\ndpbxTR78DEA0jUwTlpWVPACAQlXcDwi/cjbpCN4jgeU8rMaGJ6hsSBK6OadWD5pV\ns7LyP/mejyHR9DzM00nHuChD1/5PbirPIJVybaxNSAt8Lp9HhP1XZ0MNuGaD9T7Q\nGuvNnFLhsTVv546C7Rp1+P4Afi4jSz2ieDVUtqxQ30w5rOGRuPmsx27D/yqd68Lt\nENFmaivByh431XDjUj4N81rhULfWTa0emYkkuUKzeyHIyZnbfzneTdtNr+QS4SZE\n/Ken/iIJ3Bd+Tp+7rGYo55cCAwEAATANBgkqhkiG9w0BAQsFAAOCAQEAXzn+HAkm\nnieBggdcekm2rqc1jmOsuSujt97IwvqbjqF46MZo50OSxlehdx3Qe4lvnn0L8wJI\n6M+hrW87DGDZygNchb8hW7ttFhN2l9y9Ug7rYflTONzxYG4WjunIGH5Y42pSjIlU\nuBL2mWDTQCq5g1Pn4X8vH2TPLduoiy3XTc1b9E48TJ9ZE

In [61]:
verify_chain(client2_cert)


subjective: client2.com 	 issuer: ca2_1
====check revoke status for client2.com from ca2_1
b'{"msg":"Not revoke"}'
=====verfy cert using ca2_1 public key
correct signature

subjective: ca2_1 	 issuer: ca2
====check revoke status for ca2_1 from ca2
b'{"msg":"Not revoke"}'
=====verfy cert using ca2 public key
correct signature

subjective: ca2 	 issuer: ca_root
====check revoke status for ca2 from ca_root
b'{"msg":"Already revoke"}'
=====verfy cert using ca_root public key
correct signature


In [62]:
# openssl req -newkey rsa:2048 -keyout PRIVATEKEY.key -out MYCSR.csr

f=open('MYCSR.csr','rb')
openssl_csr=x509.load_pem_x509_csr(f.read())
print(openssl_csr.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
response = requests.post(f"http://127.0.0.1:{ca_ports['ca1_1']}/issue_cert",files={'csr_file':openssl_csr.public_bytes(serialization.Encoding.PEM)})
client2_cert = x509.load_pem_x509_certificate(response.content)
print("applicant: "+client2_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+client2_cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client2_cert.not_valid_before)
print(client2_cert.not_valid_after)

openssl
applicant: openssl
issuer: ca1_1
2021-11-27 08:13:42
2021-12-07 08:13:42
